In [1]:
#Author: Miguel S. Chique Sayre / Resource and Modeling Geologist

import pyodbc
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
#SQL SERVER Database
conn_sql = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-VMEQK3V\SQLEXPRESS;'
                      'Database=GVDATA1;'
                      'Trusted_Connection=yes;')

GvM_Collar = pd.read_sql_query('SELECT * FROM DATSSONDAJES', conn_sql)
GvM_Survey = pd.read_sql_query('SELECT * FROM DATSSURVEY', conn_sql)

GvM_Lit = pd.read_sql_query('SELECT * FROM GvM_Lit', conn_sql)
GvM_Lit["Lenght_m"]= round(GvM_Lit["Lenght_m"],2)

GvM_Alt = pd.read_sql_query('SELECT * FROM GvM_Alt', conn_sql)
GvM_Alt["Lenght_m"]= round(GvM_Alt["Lenght_m"],2)

GvM_Min = pd.read_sql_query('SELECT * FROM GvM_Min', conn_sql)
GvM_Min["Lenght_m"]= round(GvM_Min["Lenght_m"],2)
GvM_Min=GvM_Min.sort_values(['Hole_ID','Depth_From'])

GvM_Est = pd.read_sql_query('SELECT * FROM GvM_Est', conn_sql)
GvM_Est["Lenght_m"]= round(GvM_Est["Lenght_m"],2)

In [3]:
#Configuración de campos del Collar y Survey para exportación

GvM_Collar_Filter=GvM_Collar[["IDSONDAJE","ESTE","NORTE","COTACOLLAR","LARGO","OBSERVACIONES","FECHA_INICIO",
                             "FECHA_TERMINO","SECTOR"]]
GvM_Collar_Filter=GvM_Collar_Filter.rename(columns={'IDSONDAJE':'Hole_ID','ESTE':'East','NORTE':'North','COTACOLLAR':'Elevation'
                                                    ,'LARGO':'Depth_Total','OBSERVACIONES':'Estate',
                                                    'FECHA_INICIO':'Date_Start','FECHA_TERMINO':'Date_End','SECTOR':'Zone'})

GvM_Survey_Filter=GvM_Survey[["IDSONDAJE","PROFUNDIDAD","AZIMUTH","INCLINACION"]]
GvM_Survey_Filter=GvM_Survey_Filter.rename(columns={'IDSONDAJE':'Hole_ID','PROFUNDIDAD':'Depth','AZIMUTH':'Azimuth'
                                                    ,'INCLINACION':'Dip'})

Collar_GvM_Aux = GvM_Collar_Filter[["Hole_ID","Zone"]]
GvM_Survey_Filter = pd.merge(GvM_Survey_Filter, Collar_GvM_Aux, on='Hole_ID')

In [4]:
#Guardado del compilado general

writer = pd.ExcelWriter('GvM_DB_Compilado.xlsx')

GvM_Collar_Filter.to_excel(writer, sheet_name="Collar", index=False)
GvM_Survey_Filter.to_excel(writer, sheet_name="Survey", index=False)
GvM_Lit.to_excel(writer, sheet_name="Lit", index=False)
GvM_Alt.to_excel(writer, sheet_name="Alt", index=False)
GvM_Min.to_excel(writer, sheet_name="Min", index=False)
GvM_Est.to_excel(writer, sheet_name="Est", index=False)

writer.save()
writer.close()